Scatterplot

In [ ]:
##data preprocessing
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

raw_data = Dataset("20210701_ny-alesund_iwc-Z-T-method.nc")
data = raw_data.variables["iwc"]
data_np = np.transpose(data[:])

#number of time-/height-steps
time = np.size(data_np,1)
height = np.size(data_np,0)

#data max/min
data_max = np.amax(data_np)
data_min = 1
for x in range(0, height):
    for y in range(0, time):
        if((data_np[x,y] < data_min) and (data_np[x,y] > 0.0)):
            data_min = data_np[x,y]


##omc generation
from omccolors import omccolors

cmap = "viridis"

(colormap, min_exp, max_exp) = omccolors.generate_omc(data_min, data_max, cmap)


##create matplotlib scatterplot

#creating grid
x = np.arange(time)
y = np.arange(height)
x, y = np.meshgrid(x,y)

#creating the scatterplot
fig = plt.figure(figsize=(30, 10))

plt.scatter(
    x = x,
    y = y,
    c = data_np[y,x],
    s = 1,
    linewidths = 0, 
    cmap = colormap, #generated omc colormap
    norm = matplotlib.colors.LogNorm() #logarithmic normalization
)

matplotlib.pyplot.rc('axes', linewidth=2)
plt.title('Ice water content - 20210701', fontsize = 25)
plt.xlabel('Time (UTC)', fontsize = 25)
plt.ylabel('Height (km)', fontsize = 25)
plt.yticks(list(range(0, height+2, int(height/12))), list(range(0, 13)), fontsize = 25)
plt.xticks(list(range(0, time+2, int(time/6))), ["00:00", "04:00", "08:00", "12:00", "16:00", "20:00", "00:00"], fontsize = 25)
plt.xlim([0, time+2])
plt.ylim([0, height+2])
cbar = plt.colorbar()
cbar.set_label(label = "$kg/m^3$", fontsize = 25)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(25)
plt.clim(10**min_exp, 10**max_exp)

plt.show()

Choropleth

In [ ]:
##data preprocessing
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px
from urllib.request import urlopen
import json

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

df["value"] = np.random.randint(1, 100000, df.shape[0])

data_min = df["value"].min()
data_max = df["value"].max()


##omc generation
from omccolors import omccolors

cmap = "viridis"

(colormap, min_exp, max_exp) = omccolors.generate_omc(data_min, data_max, cmap)


##generate choropleth

cmap = matplotlib.cm.get_cmap(colormap)
cmap_rgb = []
norm = matplotlib.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = matplotlib.colors.colorConverter.to_rgb(cmap(norm(i)))
       cmap_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

cmap = matplotlib_to_plotly(cmap, 255)

fig = px.choropleth(df, geojson=counties, locations='fips', color=np.log10(df["value"]),
                        color_continuous_scale = cmap,
                        range_color=(np.floor(np.min(np.log10(df["value"]))),np.max(np.log10(df["value"]))),
                        scope="usa",
                        labels={'color':'log10(value)'},
                        hover_name = 'value'
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Value', tickprefix='1.e'))#, x = 0.62))
fig.show()